In [3]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
import re
import itertools as it
import nltk as nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
# from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [4]:
df = pd.read_csv('reviews_hotels.csv')

In [4]:
df

,Unnamed: 0,hotel,commentaire,note
0,0,Appi Hotel,Appi hôtel à éviter absolument. Chambre pas fa...,1
1,1,Appi Hotel,Horrible ne jamais y mettre les pieds...a bani...,1
2,2,Appi Hotel,"Hotel bruyant, miteux qu'il faut fuir !!! Douc...",1
3,3,Appi Hotel,J’ai réservé plus de 4 mois à l’avance et lors...,1
4,4,Appi Hotel,Hôtel dont les chambres sont confortables et a...,0
...,...,...,...,...
53933,11136,Hotel_Relais_Bosquet_Paris,Nous avons adoré le Rancho Bernardo ! Après de...,0
53934,11137,Hotel_Relais_Bosquet_Paris,Mon mari et moi-même avons séjourné à l'hôtel ...,0
53935,11138,Hotel_Relais_Bosquet_Paris,Ma fille et moi-même rentrons de notre séjour ...,0
53936,11139,Hotel_Relais_Bosquet_Paris,Mon mari et moi avons séjourné dans cet hôtel ...,0


In [5]:
df = df.drop(['Unnamed: 0'], axis = 1)


In [6]:
df

,hotel,commentaire,note
0,Appi Hotel,Appi hôtel à éviter absolument. Chambre pas fa...,1
1,Appi Hotel,Horrible ne jamais y mettre les pieds...a bani...,1
2,Appi Hotel,"Hotel bruyant, miteux qu'il faut fuir !!! Douc...",1
3,Appi Hotel,J’ai réservé plus de 4 mois à l’avance et lors...,1
4,Appi Hotel,Hôtel dont les chambres sont confortables et a...,0
...,...,...,...
53933,Hotel_Relais_Bosquet_Paris,Nous avons adoré le Rancho Bernardo ! Après de...,0
53934,Hotel_Relais_Bosquet_Paris,Mon mari et moi-même avons séjourné à l'hôtel ...,0
53935,Hotel_Relais_Bosquet_Paris,Ma fille et moi-même rentrons de notre séjour ...,0
53936,Hotel_Relais_Bosquet_Paris,Mon mari et moi avons séjourné dans cet hôtel ...,0


In [6]:
def com_preproc(commentaires):
    ponctuations = '''!()-[]{};:'"\,./?@#$%^&@*_~'''
    pas_de_ponc = ""
    for carac in commentaires:
        if carac not in ponctuations:
            pas_de_ponc = pas_de_ponc + carac
    mots = stopwords.words('french')
    comm_nettoyés = ""
    for mot in pas_de_ponc.split():
        if mot not in mots:
            comm_nettoyés = comm_nettoyés + ' ' + mot
    return comm_nettoyés    

In [7]:
df["commentaire"] = df["commentaire"].apply(com_preproc)

In [8]:

nombre_évaluation = df["note"].value_counts()
nombre_évaluation

0    40250
1    13688
Name: note, dtype: int64

In [9]:
messages_bow = CountVectorizer(analyzer=com_preproc).fit_transform(df['commentaire'])

In [10]:
text = " ".join(commentaires for commentaires in df.commentaire)
print ("Il y a {} de mots dans les commentaires pris.".format(len(text)))

Il y a 18022532 de mots dans les commentaires pris.


In [11]:
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['note'], test_size = 0.20, random_state = 0)

In [12]:
multiNB = MultinomialNB()
multiNB.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
#Print the predictions
print(multiNB.predict(X_train))#Print the actual values
print(y_train.values)

[0 1 0 ... 1 0 0]
[0 0 0 ... 1 0 0]


In [14]:
#Print the predictions
print('valeurs prédites: ',multiNB.predict(X_test))#Print Actual Label
print('------------------------------------------------------------------------------------')
print('réelles valeurs: ',y_test.values)

valeurs prédites:  [1 1 0 ... 1 0 0]
------------------------------------------------------------------------------------
réelles valeurs:  [1 1 0 ... 1 0 0]


In [15]:
import pickle
# save the model to disk
filename = 'finalized_model.pkl'
pickle.dump(multiNB, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.7427697441601779


In [16]:
accuracy_score
pred = multiNB.predict(X_test)
print(classification_report(y_test ,pred ))
print('--------------------------------------')
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print('-------------------------------------')
print('Accuracy: ', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.80      0.87      0.84      8034
           1       0.49      0.36      0.41      2754

    accuracy                           0.74     10788
   macro avg       0.65      0.62      0.63     10788
weighted avg       0.72      0.74      0.73     10788

--------------------------------------
Confusion Matrix: 
 [[7029 1005]
 [1770  984]]
-------------------------------------
Accuracy:  0.7427697441601779
